# Urban growth modeling in GRASS GIS: Parallel computing case study
The purpose of this notebook is to demonstrate several parallel computing principles and how they are implemented in GRASS GIS.
We use FUTURES urban growth model implemented as a GRASS GIS addon [r.futures](https://grass.osgeo.org/grass-stable/manuals/addons/r.futures.html).

This notebook uses a [prepared dataset](https://doi.org/10.5281/zenodo.6577922). This dataset is a GRASS GIS Location containing:
 * [NLCD 2001-2019](https://www.mrlc.gov/) (land use and impervious surface descriptor)
 * [US county boundaries](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)
 * [US-PAD protected areas](https://www.usgs.gov/programs/gap-analysis-project/science/pad-us-data-overview)
 * [USGS DEM](https://www.usgs.gov/3d-elevation-program/about-3dep-products-services)
 
Population files were downloaded from [Zenodo](https://doi.org/10.5281/zenodo.6577903).

All the required software is already installed and includes:
 * _GRASS GIS v8.2_ with the following addons: _r.futures, r.mapcalc.tiled, r.sample.category_
 * _R_ with the following packages: _lme4, optparse, MuMIn, snow_
 * _GNU Parallel_
 * _Python 3_ with packages _pandas_
 
This notebook combines Python 3 and Bash cells. By default a code cell is in Python. We use IPython [cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics) including `%%bash`, `%%time`, and `%%writefile`.

## Setting up
The data structure is set up this way
```
├── grassdata
|   └── FUTURES_SE_USA (location)
│       ├── PERMANENT  (mapset)
│       └── tutorial   (mapset)
├── observed_population_SE_counties_2001-2019.csv
└── projected_population_SE_counties_2020-2100_SSP2.csv

```

Change the current directory to wherever you unzipped the data, example below (modify as needed):



In [ ]:
import os

os.chdir(os.path.expanduser("~/data"))

Import Python packages and initialize GRASS GIS session:

In [ ]:
import subprocess
import sys
import pathlib
import json
import pandas as pd
from IPython.display import Image

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("~/data/grassdata/", "FUTURES_SE_USA", "tutorial")

## Data preprocessing
List dataset layers:

In [ ]:
%%bash
g.list type=raster,vector -p

### Process county boundaries
Extract 3 counties in Southeast US states (Georgia, North Carolina, South Carolina)
and convert GEOID columns to integer, which is going to be useful for further processing.

In [ ]:
%%bash
v.extract tl_2021_us_county output=counties where="STATEFP in ('13', '37', '45')" --q
v.db.addcolumn counties column="state integer" --q
v.db.addcolumn counties column="county integer" --q
v.db.update counties col=state qcol="CAST(STATEFP AS integer)" --q
v.db.update counties col=county qcol="CAST(GEOID AS integer)" --q

In [ ]:
m = gj.InteractiveMap()
m.add_vector(name="counties")
m.show()

Split and rasterize states for further parallelization steps:

In [ ]:
states = [13, 37, 45]
gs.use_temp_region()
for state in states:
    gs.run_command(
        "v.extract",
        input="SE_counties",
        where=f"state == '{state}'",
        output=f"state_{state}",
    )
    gs.run_command("g.region", vector=f"state_{state}", align="nlcd_2019")
    gs.run_command(
        "v.to.rast",
        input=f"state_{state}",
        output=f"state_{state}",
        use="attr",
        attribute_column="county",
    )
gs.del_temp_region()

In [ ]:
m = gj.Map()
m.d_rast(map="state_37")
m.show()